# ResNet (Residual Network)

In [1]:
import collections

import chainer
from chainer import functions as F
from chainer import initializers
from chainer import links as L

## モデル

ResNetは画像識別タスクで高いパフォーマンスを出したモデルである。  
VGG16などよりもさらに層の深いモデルである。ResNetでは以下の残差ブロックと呼ばれる構造を導入した。

<img src="image/block.png", style="width: 100px;">
(引用 「Deep Residual Learning for Image Recognition」)

このブロックでは、畳込み層の出力と入力の和を出力する。  
この残差ブロックを複数組み合わせたモデルがResNetであり、以下は34層のResNet全体の構造である。

<img src="image/resnet.png", style="width: 100px;">
(引用 「Deep Residual Learning for Image Recognition」)

In [2]:
class BasicA(chainer.Chain):
    # (特徴マップの)sizeが変わるときの残差ブロック
    def __init__(self, in_size, ch, stride):
        super(BasicA, self).__init__()
        # 重みの初期値
        w = initializers.HeNormal()

        with self.init_scope():
            # sizeが半分に
            self.conv1 = L.Convolution2D(in_size, ch, ksize=3, 
                                         stride=stride, pad=1, initialW=w, nobias=True)
            self.bn1 = L.BatchNormalization(ch)      
            self.conv2 = L.Convolution2D(ch, ch, ksize=3, 
                                         stride=1, pad=1, initialW=w, nobias=True)
            self.bn2 = L.BatchNormalization(ch)
            # sizeが半分に
            self.conv3 = L.Convolution2D(in_size, ch, ksize=3, 
                                         stride=stride, pad=1, initialW=w, nobias=True)
            self.bn3 = L.BatchNormalization(ch)

    def __call__(self, x):
        h1 = F.relu(self.bn1(self.conv1(x)))
        h1 = self.bn2(self.conv2(h1))
        
        h2 = self.bn3(self.conv3(x))

        return F.relu(h1 + h2)


class BasicB(chainer.Chain):
    # 普通の残差ブロック
    def __init__(self, in_size, ch):
        super(BasicB, self).__init__()
        # 重みの初期値
        w = chainer.initializers.HeNormal()

        with self.init_scope():
            self.conv1 = L.Convolution2D(in_size, ch, ksize=3, 
                                         stride=1, pad=1, initialW=w, nobias=True)
            self.bn1 = L.BatchNormalization(ch)
            self.conv2 = L.Convolution2D(ch, ch, ksize=3, 
                                         stride=1, pad=1, initialW=w, nobias=True)
            self.bn2 = L.BatchNormalization(ch)

    def __call__(self, x):
        h = F.relu(self.bn1(self.conv1(x)))
        h = self.bn2(self.conv2(h))

        return F.relu(h + x)


class BasicBlock(chainer.ChainList):
    """
    残差ブロックを並べたもの。
    残差ブロックA１個と残差ブロックB複数個
    """
    def __init__(self, layer, in_size, ch, stride=2):
        super(BasicBlock, self).__init__()
        with self.init_scope():
            self.add_link(BasicA(in_size, ch, stride))
            for i in range(1, layer):
                self.add_link(BasicB(ch, ch))

    def __call__(self, x):
        for f in self.children():
            x = f(x)
        return x


class ResNet34(chainer.Chain):
    # 34層のResNet
    def  __init__(self, n_classes=1000):
        super(ResNet34, self).__init__()
        
        with self.init_scope():
            self.conv1 = L.Convolution2D(3, 64, ksize=7, stride=2, pad=3, 
                                         initialW=initializers.HeNormal(), nobias=True)
            self.bn1 = L.BatchNormalization(64)           
            self.res2 = BasicBlock(3, 64, 64, stride=1)            
            self.res3 = BasicBlock(4, 64, 128)     
            self.res4 = BasicBlock(6, 128, 256)
            self.res5 = BasicBlock(3, 256, 512)            
            self.fc = L.Linear(512, n_classes)
             
    def __call__(self, x):                # size: 224
        h = self.bn1(self.conv1(x))  # size: 112
        h = F.max_pooling_2d(F.relu(h), ksize=3, stride=2)     # size: 56
        h = self.res2(h)                   # size: 56
        h = self.res3(h)                   # size: 28
        h = self.res4(h)                   # size: 14
        h = self.res5(h)                   # size: 7
        h = F.average_pooling_2d(h, 7, stride=1)       # size: 1
        h = self.fc(h)

        return h

In [3]:
resnet = ResNet34()

In [4]:
import numpy as np
img = np.ones((1, 3, 224, 224), dtype=np.float32)
output = resnet(img)
print(output.shape)

(1, 1000)
